In [1]:
import gym
import numpy as np
import tensorflow as tf

from my_rl.agent import Agent, ReplayBuffer, Algorithm
from my_rl.algorithms.sac import SAC

d:\113403\project\DQL\venv\lib\site-packages\tensorflow\__init__.py:29: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
d:\113403\project\DQL\venv\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
d:\113403\project\DQL\venv\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
d:\113403\project\DQL\venv\lib\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
d:\113403\project

In [2]:
game_name = 'BipedalWalker-v3'

# create env of the gam
env = gym.make(game_name, render_mode=None)
state_space = env.observation_space.shape[0]
action_space = env.action_space.shape[0]
action_scale = (env.action_space.high[0] - env.action_space.low[0])/2

# create agent
rb = ReplayBuffer(capacity=10000)
alg = Algorithm(algorithm=SAC(action_dims=action_space))
agent = Agent(replay_buffer=rb, algorithm=alg, action_scale=action_scale, state_space=state_space, action_space=action_space)

d:\113403\project\DQL\venv\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
d:\113403\project\DQL\venv\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
agent.replay_buffer.clear_memory()

for episode in range(1000):
    observation, _ = env.reset(return_info=True)
    al_list = []
    reward_list = []

    for step in range(1000):
        action = agent.sample_action(observation.reshape(1, state_space))
        new_observation, reward, done, info = env.step(action)
        reward_list.append(reward)
        done = tf.cast(done, tf.float32)
        agent.replay_buffer.add_data((tf.squeeze(observation), action, reward, done, tf.squeeze(new_observation)))

        if agent.replay_buffer.enough_data:
            loss = agent.train()
            al_list.append(loss)

        observation = new_observation.copy()

        if done:
            break

    # report losses after every episode
    if episode%1 == 0 and agent.replay_buffer.enough_data:
        print(f'episode:{episode}, avg_al={tf.reduce_mean(al_list)} sum_r={tf.reduce_sum(reward_list)}')


env.close()

d:\113403\project\DQL\my_rl\agent.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([self.memory[x] for x in batch_index])


episode:47, avg_al=1.084346890449524 sum_r=-143.51987276325644
episode:48, avg_al=0.5811080932617188 sum_r=-95.50332801712057
episode:49, avg_al=0.5559979677200317 sum_r=-119.47039509302408
episode:50, avg_al=0.6238800883293152 sum_r=-98.98048688071097
episode:51, avg_al=0.597909152507782 sum_r=-101.4786672308746
episode:52, avg_al=0.644525945186615 sum_r=-98.5977357846008
episode:53, avg_al=0.7515549659729004 sum_r=-98.46859999090806
episode:54, avg_al=0.7453850507736206 sum_r=-121.41882365320637
episode:55, avg_al=0.929786205291748 sum_r=-100.10541974299727


In [ ]:
env.close()